In [1]:
import jupyter_black
import black
import pandas as pd
import gc
from handlers import *

gc.collect()

jupyter_black.load(
    lab=False,
    line_length=100,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

DEBUG:jupyter_black:config: {'line_length': 100, 'target_versions': {<TargetVersion.PY310: 10>}}


<IPython.core.display.Javascript object>

In [2]:
# Retrieve only the most recent data from pdt_capacity

retrieve_most_recent_pdt_capacity(dataset_id="v_extract1", table_id="pdt_capacity")

In [2]:
pdt_capacity = read_bigquery_table(dataset_id="preprocessed", table_id="pdt_capacity", key_field="PRODUCTCODE")

In [7]:
pdt_incexcl = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_incexcl", key_field="PRODUCTCODE")

In [9]:
pdt_inclexcl = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_inclexcl", key_field="PRODUCTCODE")

In [10]:
pdt_inclexcl_ENG = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_inclexcl_ENG", key_field="PRODUCTCODE")

In [11]:
pdt_itinerary = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_itinerary", key_field="PRODUCTCODE")

In [12]:
pdt_product_detail = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_product_detail", key_field="PRODUCTCODE")

In [13]:
pdt_product_level = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_product_level", key_field="PRODUCTCODE")

In [14]:
pdt_reviews = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_reviews", key_field="PRODUCTCODE")

In [15]:
pdt_tags = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_tags", key_field="PRODUCTCODE")

In [16]:
pdt_tourgrades = read_bigquery_table(dataset_id="v_extract1", table_id="pdt_tourgrades", key_field="PRODUCTCODE")

In [17]:
tags_cleaned = read_bigquery_table(dataset_id="v_extract1", table_id="tags_cleaned", key_field="PRODUCTCODE")

In [18]:
pdt_capacity.shape

(5132, 15)

In [19]:
pdt_incexcl.shape

(11180, 6)